# Sampling probability arrays: a  Python class

When working in data science, we often need arrays of weights that sum to one, we call these probability arrays or probability vectors. 
For example, in classification tasks, we want the output to be a array of probabilities representing predicted class distributions. When implement mixture models, probability arrays can represent the weight of each component. In finance, they might define the weights of assets in a portfolio where the total allocation must sum to one.

However, ensuring that these vectors are correctly sampled and always sum to one is not trivial. A simple approach, like sampling random numbers and normalizing them, can lead to uneven or biased distributions. So, how can we guarantee that sampled arrays always meet this constraint?

One way to do so is to imagine placing beads into bins. Picture having a finite number of beads, and distributing them across five bins. Each bin represents one component of the probability array, and the fraction of beads in each bin determines the corresponding probability weight. 


<figure>
  <img src="https://github.com/PessoaP/blog/blob/master/Beads/beads1.png?raw=true" alt="S"/>
</figure>

This analogy is not only intuitive but also robust, as it naturally ensures that the total sum is preserved without needing normalization after. Additionally, this representation provides flexibility: by increasing the number of beads, we can achieve higher precision in our probability arrays. Moreover, this bead-and-bin model offers a precise way to represent probabilities. Since beads are discrete, the total count is inherently stable, avoiding the floating-point errors that can arise when dealing with continuous random variables.

In this blog post, we build a Python class called `prob_array` to model probability arrays using the bead-and-bin analogy. We'll cover three core functions: (i) initializing arrays from raw counts or by another probability vector, (ii) proposing symmetric updates by redistributing beads; and (iii) calculating the log-probability under a multinomial prior, including necessary mathematical corrections. 
By the end, you'll have a clean implementation of `prob_array`, with examples showcasing initialization, symmetric proposals, and probabilistic evaluation. Let’s begin with array initialization.

## Array initialization
First, let us creat a method for array initialization that can handle 3 types of input:

- No input array: The class initializes a balanced distribution where each component has an equal number of beads.

- Integer input: The input is assumed to represent bead counts directly.

- Probability array input: The array is scaled to match the total bead count while preserving proportions as much as possible.

This is implemented as the Python class below

In [1]:
import numpy as np
normalize = lambda x: x/x.sum()

class prob_array:
    def __init__(self,array=None,components=20,beads=10000):
        if array is None:
            self.counts = np.ones(components,dtype=int)*(beads//components)
            self.counts[:beads%components]+=1
        elif array.dtype == int:
            self.counts = array
        elif np.isclose(1.,np.sum(array)):
            self.counts = np.floor(array*beads).astype(int)
            self.counts[np.argsort(array)[:beads%self.counts.sum()]]+=1

        self.prob = normalize(self.counts)


## Proposing symmetric updates

Once we have a probability array, we may want to generate new samples that are close to the array we have while ensuring they still sum to 1. This is particularly relevant in Markov Chain Monte Carlo (MCMC) methods, such as Metropolis-Hastings, where it is convenient to use symmetric proposals as they help simplify computations. In this context, a symmetric proposal means that the probability of transitioning from state $A$ to state $B$ is equal to the probability of transitioning from $B$ to $A$. 

The `proposal` function below implements it as a bead-movement scheme. Each bead has a probability $r$ to move, being $r/2$ of moving to the left bin and $r/2$ of moving to the right bin, staying in the same place with probability $1-r$. If it is in the first bin and it moves left (or in the las bin and move right) it just stays in place
<figure>
  <img src="https://github.com/PessoaP/blog/blob/master/Beads/beads2.png?raw=true" alt="S"/>
</figure>
Note that for each bead, the movements are symmetric, meaning each bead has an equal probability of moving to a neighboring bin as a bead in that bin has of moving back, the proposal remains symmetric.


We implement a function doing this below

In [2]:
def proposal(p_array, rate=0.01):
    movables = np.random.binomial(p_array.counts, rate)  # Select beads to move
    new_counts = p_array.counts - movables 

    mvleft = np.random.binomial(movables, 0.5)  # Half move left
    mvright = movables - mvleft  # Half move right

    new_counts[:-1] += mvleft[1:]
    new_counts[0] += mvleft[0]  # Beads from the first bin stay in place

    new_counts[1:] += mvright[:-1]
    new_counts[-1] += mvright[-1]  # Beads from the last bin stay in place

    return prob_array(new_counts,new_counts.size,new_counts.sum())

This function maintains the total bead count while making small perturbations. The `rate` parameter  (equivalent to $r$) controls the fraction of beads that move at each step. 

## Evaluating Probability with a Multinomial Model

Now, suppose we want to evaluate the likelihood that a bead-and-bin configuration was generated from a multinomial distribution. This approach provides a specific way to define what [Bayesian statistics refer to as a prior](https://labpresse.com/why-do-we-need-bayesian-statistics-part-i-asserting-if-a-coin-is-biased-tutorial/), though that is beyond our focus here.

A multinomial model assumes that each bead is assigned a priori to each bin with a probability given by another probability vector, $\alpha = \{\alpha_1, alpha_2, \ldots, \alpha_n\}$, which represents the expected proportions of beads across bins. 

Given a total number of beads, the multinomial model determines the probability is given by 
$$ p(\{k_i\}|\{\alpha_i\},N) = \frac{N!}{k_1! k_2! \ldots k_I!}  \alpha_1^{k_1} \alpha_2^{k_2} \ldots \alpha_I^{k_I} $$

Where 
- $N$ is the total number of beads
- $k_i$ represents the number of beads in the $i$-th bin, with $I$ being the total number of beads (`components`)
- $\alpha_i$ the probability a bead was assigned to the $i$-th bead
With the factorial prefactor acounting for the number of ways to arrange the beads among the bins such that one still have the same $k_i$.

When dealing with probabilities working with their logarithm simplifies computations and mitigates numerical underflow. Using the property that the logarithm of a product is the sum of the logarithms, we take the log of the multinomial probability
$$ \log p(\{k_i\}|\{\alpha_i\},N) = 
\log N! - \sum_{i=1}^I  \log( k_i! ) + \sum_{i=1}^I {k_i} \log(\alpha_i)  . $$

An interesting trick at this stage is to use the [Gamma function](https://en.wikipedia.org/wiki/Gamma_function). Although much can be said about it, here we will simply note that the Gamma function is a well-known mathematical function that, for positive integers, matches the factorial as $\Gamma(n) = (n-1)!$.  Thus, instead of computing the logarithm of a factorial, we can use the logarithm of the Gamma function, which is implemented in libraries such as [Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.loggamma.html). Leading us to the function:

In [3]:
from scipy.special import loggamma
logfactorial = lambda x: loggamma(x+1)    
def multinomial_logprob(p_array,alpha):
    N = p_array.counts.sum()
    alpha = normalize(alpha)
    return logfactorial(N) - logfactorial(p_array.counts).sum() + (p_array.counts*np.log(alpha)).sum() 

## Making it a single class

For ease of use in Python we can combine all of the functionnality meantioned here into a single class that is structured and reusable. This serves as an exercise in object-oriented programming while also making it easier to handle probability arrays, generate proposals, and compute log-probabilities in a unified interface.

In [4]:
import numpy as np
from scipy.special import loggamma

# Utility functions
normalize = lambda x: x / x.sum()  # Ensures the input array sums to 1
logfactorial = lambda x: loggamma(x + 1)  # Computes the log-factorial using log-gamma

class prob_array:
    def __init__(self, array=None, components=20, beads=10000):
        """
        Initialize a probability array.
        
        Parameters:
        - array: Optional numpy array, can represent counts or probabilities.
        - components: Number of components in the probability array.
        - beads: Total number of beads (samples) for normalization.
        """
        if array is None:
            # Uniform distribution of beads across components
            self.counts = np.ones(components, dtype=int) * (beads // components)
            self.counts[:beads % components] += 1  # Distribute remaining beads
        elif array.dtype == int:
            # If array represents counts directly
            self.counts = array
        elif np.isclose(1.0, np.sum(array)):
            # If array represents probabilities
            self.counts = np.floor(array * beads).astype(int)
            remainder = beads - self.counts.sum()
            self.counts[np.argsort(array)[-remainder:]] += 1  # Adjust for rounding errors
        else:
            raise ValueError("Input array must be counts (int) or probabilities (sum to 1).")
        
        self.prob = normalize(self.counts)  # Normalize counts to probabilities

    def proposal(self, rate=0.01):
        """
        Generate a proposal for a new probability array.
        
        Parameters:
        - rate: Rate of change for beads redistribution.
        
        Returns:
        - New prob_array instance with adjusted counts.
        """
        movables = np.random.binomial(self.counts, rate)  # Determine movable beads
        new_counts = self.counts - movables

        # Redistribute beads left and right
        mvleft = np.random.binomial(movables, 0.5)
        mvright = movables - mvleft

        new_counts[:-1] += mvleft[1:]
        new_counts[0] += mvleft[0]  # Beads attempting to move left from the first index stay

        new_counts[1:] += mvright[:-1]
        new_counts[-1] += mvright[-1]  # Beads attempting to move right from the last index stay

        # Ensure counts remain valid
        if np.any(new_counts < 0):
            raise ValueError("Invalid move: Negative counts detected.")
        
        return prob_array(new_counts, new_counts.size, new_counts.sum())
    
    def multinomial_logprob(self, alpha):
        """
        Compute the log-probability of the current counts given a multinomial distribution.
        
        Parameters:
        - alpha: probability assigned to each bin
        
        Returns:
        - Log-probability value.
        """
    
        N = self.counts.sum()
        alpha = normalize(alpha)
        return logfactorial(N) - logfactorial(self.counts).sum() + (self.counts*np.log(alpha)).sum() 

    def __repr__(self):
        return f"prob_array(counts={self.counts}, prob={self.prob})"


## Is this trully necessary?

<figure>
  <img src="https://i.kym-cdn.com/entries/icons/original/000/028/596/dsmGaKWMeHXe9QuJtq_ys30PNfTGnMsRuHuo_MUzGCg.jpg" alt="S" width="320"/>
</figure>

Not really.


While sampling probability vectors is important in data science, alternatives exist, such as using the [Dirichlet distribution](https://en.wikipedia.org/wiki/Dirichlet_distribution), whose sampler is [already implemented in `numpy`](https://numpy.org/doc/2.1/reference/random/generated/numpy.random.dirichlet.html). Dirichlet samples are guaranteed to be valid probability vectors while using only continuous mathematics rather than the bead discretization here.

Moreover, while symmetric proposals are often used in Metropolis-Hastings, they are not strictly required as long as you can calculate the probability of sampling from the proposal distribution.

Therefore, the Python class developed here, while pedagogically interesting, does not solve a fundamental problem in data science. I have to admit, I originally wrote this years ago as an unnecessary but intriguing attempt to reinvent the wheel. Still, I’m sharing it here in case someone finds it useful or simply enjoyable to explore.
